In [46]:
%load_ext autoreload
%autoreload 2
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:

vqe_handler = VQE(n_qubits=4, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits)

indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

Mindexed_circuit, Mresolver, Mindex_to_symbols = iid_inserter.place_identities(indexed_circuit,symbol_to_value=resolver,rate_iids_per_step=10)
Mcircuit = vqe_handler.give_unitary(Mindexed_circuit, Mresolver)

model = QNN(symbols=list(Mresolver.keys()), observable=vqe_handler.observable)
print(model(circuit), model(Mcircuit))

OMenergy, OMresolver, OMhistory = vqe_handler.vqe(Mindexed_circuit, symbols_to_values=Mresolver)
OMcircuit = vqe_handler.give_unitary(Mindexed_circuit, OMresolver)

print(model(Mcircuit), model(OMcircuit))


simplifier = Simplifier(n_qubits=vqe_handler.n_qubits)
Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.reduce_circuit(Mindexed_circuit, OMresolver, Mindex_to_symbols)

tf.Tensor([0.8191006], shape=(1,), dtype=float32) tf.Tensor([-7.9002028], shape=(1,), dtype=float32)
tf.Tensor([-7.9002028], shape=(1,), dtype=float32) tf.Tensor([-8.99995], shape=(1,), dtype=float32)


In [48]:
Scircuit = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
print(model(Scircuit),OMenergy)

tf.Tensor([-8.999951], shape=(1,), dtype=float32) tf.Tensor(-8.999948, shape=(), dtype=float32)


In [49]:
Scircuit

(0, 0): ───Rx(-0.023π)───X────────────Rz(-0.024π)───Rx(0.023π)──────────────X───
                         │                                                  │
(0, 1): ───Rx(π)─────────@────────────Rz(-0.026π)───Rx(0.167π)───Rz(0.5π)───@───

(0, 2): ───Rx(0.002π)────Rz(0.051π)───Rx(-0.002π)───X───────────────────────────
                                                    │
(0, 3): ───Rx(0.167π)────Rz(-0.5π)──────────────────@────────────Rx(-π)─────────

In [50]:
OMcircuit

(0, 0): ───Rx(0.004π)───X───Rx(-0.027π)───Rz(-0.024π)───Rx(0.023π)────X────────────
                        │                                             │
(0, 1): ───Rx(π)────────@───Rz(-0.026π)───Rx(0.167π)────Rz(0.5π)──────@────────────

(0, 2): ───Rx(0.017π)───X───Rx(-0.015π)───Rz(0.051π)────Rx(-0.002π)───X────────────
                        │                                             │
(0, 3): ────────────────@───Rz(-0.032π)───Rx(0.167π)────Rz(-0.5π)─────@───Rx(-π)───

In [45]:
0.592+0.459

1.051

In [41]:
len(Mindexed_circuit)

726